In [1]:
import os
import time
from pathlib import Path
import pandas as pd
import QuantLib as ql
import numpy as np
from datetime import datetime
from model_settings import ms
from itertools import product
from joblib import Parallel, delayed


pricing settings:
Actual/365 (Fixed) day counter
New York stock exchange calendar
compounding: continuous
frequency: annual



In [2]:
from model_settings import asian_option_pricer
help(asian_option_pricer)

Help on asian_option_pricer in module model_settings.asian_option_pricer object:

class asian_option_pricer(builtins.object)
 |  asian_option_pricer(seed=None)
 |
 |  Methods defined here:
 |
 |  __init__(self, seed=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  asian_option_price(self, s, k, r, g, w, averaging_type, n_fixings, fixing_frequency, past_fixings, kappa, theta, rho, eta, v0, calculation_datetime)
 |
 |  df_asian_option_price(self, df)
 |
 |  row_asian_option_price(self, row)
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object



# functions in package

In [3]:
def asian_option_price(s,k,r,g,w,averaging_type,n_fixings,fixing_frequency,past_fixings,kappa,theta,rho,eta,v0,calculation_datetime):
    s = float(s)
    k = float(k)
    r = float(r)
    g = float(g)
    rng = "pseudorandom" # could use "lowdiscrepancy"
    numPaths = 100000
    
    if w == 'call':
        option_type = ql.Option.Call 
    elif w == 'put':
        option_type = ql.Option.Put
    t = n_fixings*fixing_frequency
    
    calculation_date = ql.Date(calculation_datetime.day,calculation_datetime.month,calculation_datetime.year)
    
    periods = np.arange(fixing_frequency,t+1,fixing_frequency).astype(int)
    
    fixing_dates = [calculation_date + ql.Period(int(p),ql.Days) for p in periods]
    expiration_date = calculation_date + ql.Period(int(t),ql.Days)
    
    riskFreeTS = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, float(r), ql.Actual365Fixed()))
    dividendTS = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, float(g), ql.Actual365Fixed()))
    
    hestonProcess = ql.HestonProcess(riskFreeTS, dividendTS, ql.QuoteHandle(ql.SimpleQuote(s)), v0, kappa, theta, eta, rho)
    hestonModel = ql.HestonModel(hestonProcess)
    vanillaPayoff = ql.PlainVanillaPayoff(option_type, float(k))
    europeanExercise = ql.EuropeanExercise(expiration_date)
    
    if averaging_type == 'geometric':
        geometric_engine = ql.MCDiscreteGeometricAPHestonEngine(hestonProcess, rng, requiredSamples=numPaths,seed=123)
        geometricAverage = ql.Average().Geometric
        geometricRunningAccumulator = 1.0
        discreteGeometricAsianOption = ql.DiscreteAveragingAsianOption(
        	geometricAverage, geometricRunningAccumulator, past_fixings,
            fixing_dates, vanillaPayoff, europeanExercise)
        discreteGeometricAsianOption.setPricingEngine(geometric_engine)
        geometric_price = float(discreteGeometricAsianOption.NPV())
        return geometric_price
        
    elif averaging_type == 'arithmetic':
        arithmetic_engine = ql.MCDiscreteArithmeticAPHestonEngine(hestonProcess, rng, requiredSamples=numPaths)
        arithmeticAverage = ql.Average().Arithmetic
        arithmeticRunningAccumulator = 0.0
        discreteArithmeticAsianOption = ql.DiscreteAveragingAsianOption(
            arithmeticAverage, arithmeticRunningAccumulator, past_fixings, 
            fixing_dates, vanillaPayoff, europeanExercise)
        discreteArithmeticAsianOption.setPricingEngine(arithmetic_engine)
        arithmetic_price = float(discreteArithmeticAsianOption.NPV())
        return arithmetic_price
    else:
        print("invalid Asian option averaging type out of 'arithmetic' and geometric'")
        pass

In [4]:
def row_asian_option_price(row):
    return  asian_option_price(
        row['spot_price'],
        row['strike_price'],
        row['risk_free_rate'],
        row['dividend_rate'],
        row['w'],
        row['averaging_type'],
        row['fixing_frequency'],
        row['n_fixings'],
        row['past_fixings'],
        row['kappa'],
        row['theta'],
        row['rho'],
        row['eta'],
        row['v0'],
        row['calculation_date']
    )

In [5]:
def df_asian_option_price(df):
    prices = Parallel(n_jobs=-1)(delayed(row_asian_option_price)(row) for _, row in df.iterrows())
    return prices

# example usage

In [6]:
calculation_datetime = datetime.today()
"""
365.41	548	 0.04	0.0	put	geometric	90	10	0	0.412367	0.17771	-0.582856	0.785592	0.08079	2024-10-18 13:38:09.345440	900	147.330432	156.641565
"""
r = 0.04
g = 0.0

w = 'put'

past_fixings = 0
s,k = 365.41, 548
# s = 365.41
# k = 370.00
kappa = 0.412367
theta = 0.17771
rho = -0.582856
eta = 0.785592
v0 = 0.08079
n_fixings = 10
fixing_frequency = 90

averaging_type = 'geometric'

t = n_fixings*fixing_frequency


price = asian_option_price(s,k,r,g,w,averaging_type,n_fixings,fixing_frequency,past_fixings,kappa,theta,rho,eta,v0,calculation_datetime)
package_price = asian_option_pricer.asian_option_price(s,k,r,g,w,averaging_type,n_fixings,fixing_frequency,past_fixings,kappa,theta,rho,eta,v0,calculation_datetime)
vanilla = ms.ql_heston_price(s,k,t,r,g,w,kappa,theta,rho,eta,v0,datetime.today())

print(f"asian: {price}  {package_price}\nvanilla: {vanilla}")

asian: 155.541142583563  155.5411467754231
vanilla: 147.33043221165957


In [7]:
fixing_frequencies = [
    1,7,
    30,
    90
]
n_fixings = [
    5,
    10
]
K = np.unique(np.linspace(s*0.5,s*1.5,50).astype(int))

W = [
    'call',
    'put'
]
types = [
    'arithmetic',
    'geometric'
]
past_fixings = [0]
features = pd.DataFrame(
    product(
        [s],
        K,
        [r],
        [g],
        W,
        types,
        fixing_frequencies,
        n_fixings,
        past_fixings,
        [kappa],
        [theta],
        [rho],
        [eta],
        [v0],
        [calculation_datetime]
    ),
    columns = [
        'spot_price','strike_price','risk_free_rate','dividend_rate','w',
        'averaging_type','fixing_frequency','n_fixings','past_fixings',
        'kappa','theta','rho','eta','v0','calculation_date'
    ]
)
features['days_to_maturity'] = features['n_fixings']*features['fixing_frequency']

In [8]:
# start = time.time()
# features['asian_price'] = features.apply(asian_option_pricer.row_asian_option_price,axis=1)
# end = time.time()
# print("pandas apply cpu:")
# print(end-start)
# print()

start= time.time()
features['asian_price'] = asian_option_pricer.df_asian_option_price(features)
end = time.time()
print("parallelized cpu:")
print(end-start)

parallelized cpu:
60.39852023124695


In [9]:
features

,spot_price,strike_price,risk_free_rate,dividend_rate,w,averaging_type,fixing_frequency,n_fixings,past_fixings,kappa,theta,rho,eta,v0,calculation_date,days_to_maturity,asian_price
0,365.41,182,0.04,0.0,call,arithmetic,1,5,0,0.412367,0.17771,-0.582856,0.785592,0.08079,2024-10-18 17:01:33.483767,5,183.446279
1,365.41,182,0.04,0.0,call,arithmetic,1,10,0,0.412367,0.17771,-0.582856,0.785592,0.08079,2024-10-18 17:01:33.483767,10,183.521194
2,365.41,182,0.04,0.0,call,arithmetic,7,5,0,0.412367,0.17771,-0.582856,0.785592,0.08079,2024-10-18 17:01:33.483767,35,183.627812
3,365.41,182,0.04,0.0,call,arithmetic,7,10,0,0.412367,0.17771,-0.582856,0.785592,0.08079,2024-10-18 17:01:33.483767,70,183.774352
4,365.41,182,0.04,0.0,call,arithmetic,30,5,0,0.412367,0.17771,-0.582856,0.785592,0.08079,2024-10-18 17:01:33.483767,150,183.483922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,365.41,548,0.04,0.0,put,geometric,7,10,0,0.412367,0.17771,-0.582856,0.785592,0.08079,2024-10-18 17:01:33.483767,70,180.074227
1596,365.41,548,0.04,0.0,put,geometric,30,5,0,0.412367,0.17771,-0.582856,0.785592,0.08079,2024-10-18 17:01:33.483767,150,177.581605
1597,365.41,548,0.04,0.0,put,geometric,30,10,0,0.412367,0.17771,-0.582856,0.785592,0.08079,2024-10-18 17:01:33.483767,300,172.801803
1598,365.41,548,0.04,0.0,put,geometric,90,5,0,0.412367,0.17771,-0.582856,0.785592,0.08079,2024-10-18 17:01:33.483767,450,168.779317
